In [111]:
import requests # library to handle requests
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import pandas as pd
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy import distance
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Define reference address and destination address

In [112]:
address_ref = 'Stuttgart'
address_dest='Berlin'
search_radius=15


geolocator_ref = Nominatim(user_agent="foursquare_agent")
location_ref = geolocator_ref.geocode(address_ref)
latitude_ref = location_ref.latitude
longitude_ref = location_ref.longitude
print(latitude_ref, longitude_ref)

geolocator_dest = Nominatim(user_agent="foursquare_agent")
location_dest = geolocator_dest.geocode(address_dest)
latitude_dest = location_dest.latitude
longitude_dest = location_dest.longitude
print(latitude_dest, longitude_dest)

48.7784485 9.1800132
52.5170365 13.3888599


## Get German Address and Cooridnate Data from opendatasoft.com

In [113]:
url = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=postleitzahlen-deutschland&q=&rows=10000&facet=note&facet=plz'
results = requests.get(url).json()

In [114]:
type(results)
res1=results['records']
dataframe = json_normalize(res1)
dataframe=dataframe[['fields.note','fields.plz','fields.geo_point_2d']]
dataframe.columns=['city','post code', 'coordinates']
print(dataframe.shape)
dataframe.dropna(inplace=True)
print(dataframe.shape)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


(8699, 3)
(8697, 3)


## Filter only cities which are close to the destination area

In [115]:
coordinates=dataframe['coordinates'].to_list()
dist=[]
lat=[]
long=[]
for coor in coordinates:
    dist.append(distance.distance([latitude_dest, longitude_dest],coor).km)
    lat.append(coor[0])
    long.append(coor[1])
dataframe['distance']=dist
dataframe['latitude']=lat
dataframe['longitude']=long
destination_data=dataframe[dataframe['distance']<search_radius]
destination_data.reset_index(inplace=True)
print(destination_data.shape)
destination_data.head()

(183, 7)


,index,city,post code,coordinates,distance,latitude,longitude
0,10,Berlin Westend,14052,"[52.515925196771384, 13.256069555210905]",9.015298,52.515925,13.256070
1,31,Berlin Neukölln,12057,"[52.468382478984736, 13.46318165105761]",7.402348,52.468382,13.463182
2,68,Berlin Friedrichshain,10249,"[52.52381194391551, 13.442878223142598]",3.743409,52.523812,13.442878
3,247,Berlin Französisch Buchholz,13127,"[52.61880913270147, 13.433471475808405]",11.722082,52.618809,13.433471
4,269,Berlin,12307,"[52.388808935596224, 13.39020168220143]",14.268935,52.388809,13.390202


## Get Foursquare Data

In [116]:
CLIENT_ID = 'LWZBDYZVEVF4PTOAA25AH0HZ20XVHTTJ21QIA0J4BN0TJURE' # your Foursquare ID
CLIENT_SECRET = 'GPUKLHZYBRLLVRENUTRTZDVE1VQONPKTHGDHN5KFNYPBMP01' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LWZBDYZVEVF4PTOAA25AH0HZ20XVHTTJ21QIA0J4BN0TJURE
CLIENT_SECRET:GPUKLHZYBRLLVRENUTRTZDVE1VQONPKTHGDHN5KFNYPBMP01


In [119]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&sortByPopularity={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            sortbypop)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            print(name+' no venues generated')
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [150]:
LIMIT=20
RADIUS=500
sortbypop=1
destination_venues = getNearbyVenues(destination_data['city'],
                                   latitudes=destination_data['latitude'],
                                   longitudes=destination_data['longitude']
                                  )

Berlin Westend
Berlin Neukölln
Berlin Friedrichshain
Berlin Französisch Buchholz
Berlin
Berlin Grunewald
Berlin Lankwitz
Berlin Lichtenfelde
Berlin Lichtenfelde
Berlin Zehlendorf
Berlin Prenzlauer Berg
Berlin Neukölln
Berlin
Berlin Kreuzberg
Berlin Karlshorst
Berlin Moabit
Berlin Nikolassee
Berlin Friedrichsfelde
Berlin Rudow
Berlin
Berlin Neukölln
Berlin
Berlin Kreuzberg
Berlin Hermsdorf
Berlin Mariendorf
Berlin
Berlin Pankow
Berlin Oberschöneweide
Berlin Friedrichshain
Berlin
Berlin Alt-Hohenschönhausen
Berlin Wlhelmstadt
Berlin Prenzlauer Berg
Berlin Reinickendorf
Berlin Charlottenburg
Berlin Moabit
Berlin Lichtenberg
Berlin
Berlin Weißensee
Berlin Neukölln
Berlin-West
Berlin Blankenfelde
Berlin Reinickendorf
Berlin Steglitz
Berlin Märkisches Viertel
Berlin-West
Berlin Pankow
Berlin Kreuzberg
Berlin Schöneberg
Berlin Wartenberg
Berlin Neukölln
Berlin Heinelsdorf
Berlin Gropiusstadt
Berlin Spandau
Berlin Neukölln
Berlin Spandau
Berlin Mariendorf
Berlin Tempelhof
Berlin Spandau
Berlin

In [151]:
print(destination_venues.shape)
destination_venues.head()

(2146, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berlin Westend,52.515925,13.25607,U Neu-Westend,52.516195,13.259726,Metro Station
1,Berlin Westend,52.515925,13.25607,Westend Klause,52.516494,13.260109,Bar
2,Berlin Westend,52.515925,13.25607,Ital. Eis Gloria,52.516259,13.260992,Ice Cream Shop
3,Berlin Westend,52.515925,13.25607,Rossmann,52.515277,13.262164,Drugstore
4,Berlin Westend,52.515925,13.25607,Wiener Conditorei Caffeehaus am Steubenplatz,52.516071,13.260251,Café


## Get Venues of Reference Neighborhood

In [152]:
LIMIT=20
RADIUS=1500
sortbypop=0
reference_venues = getNearbyVenues(['Reference'],
                                   [latitude_ref],
                                   [longitude_ref]
                                  )

Reference


In [153]:
print(reference_venues.shape)
reference_venues.head()

(20, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Reference,48.778449,9.180013,Schlossplatz,48.778549,9.179855,Plaza
1,Reference,48.778449,9.180013,Landesmuseum Württemberg,48.777143,9.179542,Museum
2,Reference,48.778449,9.180013,Altes Schloss,48.777168,9.179526,History Museum
3,Reference,48.778449,9.180013,Die Bierothek,48.779198,9.178369,Beer Store
4,Reference,48.778449,9.180013,Oberer Schlossgarten,48.780002,9.182440,Park


In [155]:
all_venues=pd.concat([destination_venues, reference_venues],ignore_index=True)
print(all_venues.shape)
all_venues.head()

(2166, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berlin Westend,52.515925,13.25607,U Neu-Westend,52.516195,13.259726,Metro Station
1,Berlin Westend,52.515925,13.25607,Westend Klause,52.516494,13.260109,Bar
2,Berlin Westend,52.515925,13.25607,Ital. Eis Gloria,52.516259,13.260992,Ice Cream Shop
3,Berlin Westend,52.515925,13.25607,Rossmann,52.515277,13.262164,Drugstore
4,Berlin Westend,52.515925,13.25607,Wiener Conditorei Caffeehaus am Steubenplatz,52.516071,13.260251,Café


## Count Venues per Neighborhood

In [165]:
hoods=all_venues.groupby('Neighborhood').count()
hoods=hoods[['Venue']]
hoods.columns = ['Value Count'] 
print(hoods.shape)
print(hoods.head(69))
hood_list=hoods.index.to_list()
if 'Reference' in hood_list:
    print('Reference included')
    

(69, 1)
                             Value Count
Neighborhood                            
Berlin                               156
Berlin Alt Treptow                     5
Berlin Alt-Hohenschönhausen           29
Berlin Baumschulenweg                  9
Berlin Biesdorf                       10
...                                  ...
Berlin Zehlendorf                     21
Berlin-West                           31
Glienicke/Nordbahn                     4
Mühlenbecker Land                      4
Reference                             20

[69 rows x 1 columns]
Reference included


## Prepare Data for Clustering

In [167]:
df_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#df_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
hoods_temp= all_venues['Neighborhood'] 
df_onehot.insert(0, 'Neighborhood', hoods_temp)
# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]
print(df_onehot.shape)
df_onehot.head(2166)

(2166, 294)


,Neighborhood,ATM,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Berlin Westend,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Berlin Westend,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Berlin Westend,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Berlin Westend,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Berlin Westend,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2161,Reference,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2162,Reference,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2163,Reference,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2164,Reference,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [169]:
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
hood_list=df_grouped['Neighborhood']
if 'Reference' in hood_list:
    print('Reference included')
print(df_grouped.shape)
df_grouped.head(69)

(69, 294)


,Neighborhood,ATM,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Berlin,0.012821,0.0,0.0,0.00641,0.0,0.0,0.0,0.0,0.00,...,0.0,0.012821,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00641
1,Berlin Alt Treptow,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00000
2,Berlin Alt-Hohenschönhausen,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00000
3,Berlin Baumschulenweg,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00,...,0.0,0.111111,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00000
4,Berlin Biesdorf,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Berlin Zehlendorf,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00000
65,Berlin-West,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00,...,0.0,0.032258,0.0,0.0,0.0,0.032258,0.00,0.0,0.0,0.00000
66,Glienicke/Nordbahn,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00000
67,Mühlenbecker Land,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00000


In [172]:
df_grouped_count=pd.merge(df_grouped,hoods,on='Neighborhood')
df_grouped_filtered=df_grouped_count[df_grouped_count['Value Count']>9]
df_grouped_filtered.drop(columns='Value Count',inplace=True)
df_grouped.reset_index(inplace=True)
print(df_grouped_filtered.shape)
df_grouped_filtered.head(38)

(38, 294)


,Neighborhood,ATM,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Berlin,0.012821,0.000000,0.000000,0.00641,0.0,0.000,0.000000,0.000000,0.000000,...,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.006410
2,Berlin Alt-Hohenschönhausen,0.000000,0.000000,0.000000,0.00000,0.0,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
4,Berlin Biesdorf,0.000000,0.000000,0.000000,0.00000,0.0,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
7,Berlin Britz,0.000000,0.000000,0.000000,0.00000,0.0,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
8,Berlin Buckow,0.000000,0.000000,0.000000,0.00000,0.0,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
9,Berlin Charlottenburg,0.000000,0.000000,0.000000,0.00000,0.0,0.000,0.000000,0.000000,0.010309,...,0.010309,0.030928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
14,Berlin Friedenau,0.000000,0.000000,0.000000,0.00000,0.0,0.000,0.000000,0.000000,0.000000,...,0.000000,0.050000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000,0.000000
16,Berlin Friedrichshain,0.000000,0.000000,0.000000,0.00000,0.0,0.000,0.000000,0.000000,0.000000,...,0.000000,0.039474,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000,0.000000
17,Berlin Gesundbrunnen,0.000000,0.000000,0.000000,0.00000,0.0,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000,0.000000
20,Berlin Halensee,0.000000,0.000000,0.000000,0.00000,0.0,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
